In [1]:
! pip install evaluate
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import numpy as np
import evaluate
import datasets
import spacy
import tqdm

# Tokenizers
import re
import unicodedata
import jieba

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using", device, "device")

Using cpu device


In [3]:
dataset = load_dataset("iwslt2017", "iwslt2017-en-zh")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/231266 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8549 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/879 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 231266
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8549
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 879
    })
})

In [4]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)



In [5]:
torch.cuda.empty_cache()

## Tokenizers

## Tokenize en and zh words

In [6]:
import sentencepiece as spm
from typing import List, Union

class LangTokeniser(object):
    PAD_ID = 3  # Defined as sentencepiece custom token

    def __init__(self, lang: str, model_file=None):
        self.model = spm.SentencePieceProcessor(model_file=model_file or f"./{lang}.model")
        self.special_ids = (
            self.model.unk_id(),
            LangTokeniser.PAD_ID,  # self.model.pad_id(), # this is -1 and may give errors.
            self.model.bos_id(),
            self.model.eos_id(),
        )
        # self.vocab = None

    # def get_vocab(self):
    #     if self.vocab:
    #         return self.vocab
    #     else:
    #         vocab = {}
    #         for i in range(self.model.get_piece_size()):
    #             piece = self.model.id_to_piece(i)
    #             vocab[piece] = i
    #         self.vocab = vocab
    #         return vocab

    def __len__(self):
        return len(self.model)

    def encode_no_padding(self, sent: Union[str, List[str]], max_len=None):
        ids = self.model.encode(sent)
        if max_len is not None and len(ids) > max_len:
            ids = ids[:max_len]
        return ids

    def encode_batch(self, sents: List[str], max_len=None, padFront=False):
        return [self.encode(sent, max_len, padFront=padFront) for sent in sents]

    def encode(self, sent: Union[str, List[str]], max_len=None, padFront=False):
        if isinstance(sent, list):
            if padFront:
                return self.encode_batch(sent, max_len, padFront=padFront)
            else:
                return self.encode_batch(sent, max_len)
        ids = self.model.encode(sent)
        if max_len is not None:
            if not padFront:
                if len(ids) < max_len:
                    ids.extend([LangTokeniser.PAD_ID] * (max_len - len(ids)))
                elif len(ids) > max_len:
                    ids = ids[:max_len]
            else:
                if len(ids) < max_len:
                    # calculate padding needed
                    len_padding = max_len - len(ids)
                    ids = [LangTokeniser.PAD_ID] * len_padding + ids
                    # ids.extend([LangTokeniser.PAD_ID] * (max_len - len(ids)))
                elif len(ids) > max_len:
                    ids = ids[:max_len]

        return ids

    def decode(self, ids: List[int]):
        return self.model.decode([id for id in ids if 0 <= id < len(self) and id != LangTokeniser.PAD_ID])

    def decode_batch(self, ids: List[List[int]]):
        return [self.decode(id) for id in ids]

    def get_special_ids(self):
        UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = self.special_ids
        return UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX


class BaseBPETokeniser(object):
    """
    The class to tokenise input English sentences, and decode output Chinese Vocab IDs.

    Examples:
    ```py
    from tokenisation.sentencepiece_custom import BaseBPETokeniser

    tokeniser = BaseBPETokeniser()
    # or initialise with the model files in a separate path:
    tokeniser = BaseBPETokeniser(en_model_file="/path/to/en.model", zh_model_file="/path/to/zh.model")

    row = dataset[0]['translation']

    # Tokenise and truncate to max length of 512 for both.
    inputs = tokeniser(row['en'], text_target=row['zh'], max_len=512)
    # {
    #     'input_ids': [...],       # The English IDs
    #     'attention_mask': [...],
    #     'labels': [...]           # The Chinese IDs
    # }

    # should generate the Chinese tokens output.
    translated = tokeniser.decode(ids)

    ```
    """

    def __init__(self, en_model_file=None, zh_model_file=None):
        self.en_model = LangTokeniser("en", model_file=en_model_file)
        self.zh_model = LangTokeniser("zh", model_file=zh_model_file)

    def __len__(self):
        """
        Both the english and chinese tokenisers have the same length.
        """
        return len(self.en_model)

    def __call__(self, sent: str, text_target=None, max_len=128, max_zh_len=None):
        out = {
            "input_ids": self.en_model.encode(sent, max_len=max_len),
            "attention_mask": [1] * max_len,
        }
        if text_target:
            out["labels"] = self.zh_model.encode(
                text_target, max_len=max_zh_len or max_len
            )
        return out

    def encode_zh(self, sent: str, max_len=128):
        return self.zh_model.encode(sent, max_len=max_len)

    def encode_en(self, sent: str, max_len=128):
        return self.en_model.encode(sent, max_len=max_len)

    def encode_en_padfront(self, sent: str, max_len=128):
        return self.en_model.encode(sent, max_len=max_len, padFront=True)

    def decode_zh(self, labels: list[int]):
        return self.zh_model.decode(labels)

    def decode_zh_padfront(self, sent: str, max_len=128):
        return self.zh_model.encode(sent, max_len=max_len, padFront=True)

    def decode_zh_batch(self, labels: List[List[int]]):
        return self.zh_model.decode_batch(labels)

    def decode_en(self, labels: list[int]):
        return self.en_model.decode(labels)

    def decode_en_batch(self, labels: list[int]):
        return self.en_model.decode_batch(labels)

    def get_special_ids(self, lang: str):
        if lang == "en":
            return self.en_model.get_special_ids()
        elif lang == "zh":
            return self.zh_model.get_special_ids()

    def encode_en_no_padding(self, sent: str, max_len=None):
        return self.en_model.encode_no_padding(sent, max_len=max_len)

    def encode_zh_no_padding(self, sent: str, max_len=None):
        return self.zh_model.encode_no_padding(sent, max_len=max_len)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
# sys.path.append(os.path.abspath('../../tokenisation/sentencepiece_custom'))


In [ ]:
en_model_absolute_path = os.path.abspath('/content/drive/MyDrive/Y3S2/CS4248/PROJECT/en.model')
zh_model_absolute_path = os.path.abspath('/content/drive/MyDrive/Y3S2/CS4248/PROJECT/zh.model')

In [ ]:
tokenizer = BaseBPETokeniser(en_model_file=en_model_absolute_path, zh_model_file=zh_model_absolute_path)

In [ ]:
unk_g, pad_g, bos_g, eos_g = tokenizer.get_special_ids(lang='zh')
max_length_g = 285

def tokenize_example(example, bos_token_id=bos_g, eos_token_id=eos_g):
    en_tokens = tokenizer.encode_en_no_padding(example['translation']["en"], max_len=max_length_g)
    zh_tokens = tokenizer.encode_zh_no_padding(example['translation']["zh"], max_len=max_length_g)
    en_tokens = [bos_token_id] + en_tokens + [eos_token_id]
    zh_tokens = [bos_token_id] + zh_tokens + [eos_token_id]
    return {"en_ids": en_tokens, "zh_ids": zh_tokens}

train_data = train_data.map(tokenize_example)
valid_data = valid_data.map(tokenize_example)
test_data = test_data.map(tokenize_example)

train_data

Convert to Tensors

In [ ]:
data_type = "torch"
format_columns = ["en_ids", "zh_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

Dataloaders

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_zh_ids = [example["zh_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_zh_ids = nn.utils.rnn.pad_sequence(batch_zh_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "zh_ids": batch_zh_ids,
        }
        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 16
pad_index = pad_g

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

## Building Model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
VOCAB_LEN = len(tokenizer)

In [ ]:
input_dim = VOCAB_LEN
output_dim = VOCAB_LEN
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["en_ids"].to(device)
        trg = batch["zh_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["en_ids"].to(device)
            trg = batch["zh_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time % 60)
    return elapsed_mins, elapsed_secs

In [ ]:
CHECKPOINT_DIR = "/content/drive/MyDrive/Y3S2/CS4248/PROJECT/model_checkpoints"
# # Write content to the file
# # with open(filepath, 'w') as f:
# torch.save({"a": "yourmama"},filepath)

In [32]:
import time

patience = 3
no_improvement_epochs = 0
n_epochs = 50
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")
CHECKPOINT_DIR = ""
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

for epoch in tqdm.tqdm(range(n_epochs)):
    start_time = time.time()
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': best_valid_loss}, f"{CHECKPOINT_DIR}/model_epoch_{epoch + 1}_valid_loss_{valid_loss:.3f}.pt")
    print(f"Checkpoint saved at epoch {epoch+1}")

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Valid Loss: {valid_loss:.3f} | Valid PPL: {np.exp(valid_loss):7.3f}')
    # Auto stop
    if no_improvement_epochs >= patience:
        print(f"No improvement in val loss for {patience} consecutive epochs. Stopping training.")
        break

  2%|▏         | 1/50 [29:49<24:21:18, 1789.36s/it]

Checkpoint saved at epoch 1
Epoch: 01 | Time: 29m 44s
	Train Loss: 7.156 | Train PPL: 1282.096
	 Valid Loss: 7.514 | Valid PPL: 1833.631


  4%|▍         | 2/50 [58:05<23:07:32, 1734.42s/it]

Checkpoint saved at epoch 2
Epoch: 02 | Time: 28m 15s
	Train Loss: 6.594 | Train PPL: 731.035
	 Valid Loss: 7.280 | Valid PPL: 1451.633


  6%|▌         | 3/50 [1:24:25<21:43:19, 1663.82s/it]

...

Checkpoint saved at epoch 11
Epoch: 11 | Time: 27m 7s
	Train Loss: 5.435 | Train PPL: 229.294
	 Valid Loss: 6.959 | Valid PPL: 1053.005


 24%|██▍       | 12/50 [5:34:59<17:24:04, 1648.55s/it]

Checkpoint saved at epoch 12
Epoch: 12 | Time: 26m 56s
	Train Loss: 5.383 | Train PPL: 217.579
	 Valid Loss: 6.977 | Valid PPL: 1071.517


 24%|██▍       | 12/50 [6:02:10<19:06:53, 1810.88s/it]

Checkpoint saved at epoch 13
Epoch: 13 | Time: 27m 7s
	Train Loss: 5.335 | Train PPL: 207.507
	 Valid Loss: 6.963 | Valid PPL: 1056.702
No improvement in val loss for 3 consecutive epochs. Stopping training.


In [27]:
checkpoint = torch.load(f"{CHECKPOINT_DIR}/model_epoch_10_valid_loss_6.941.pt", map_location=torch.device('cpu'))
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)
test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 6.325 | Test PPL: 558.347 |


In [29]:
def translate_sentence(
    sentence,
    model,
    device,
    max_output_length=max_length_g,
):
    model.eval()
    with torch.no_grad():
        en_tokens = tokenizer.encode_en_no_padding(str(sentence),max_len=max_length_g)
        ids = [1] + en_tokens + [2] # It adds start and end tokens (1 and 2 respectively)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        encoder_outputs, hidden = model.encoder(tensor)
        inputs = [1]
        # attentions = torch.zeros(max_output_length, 1, len(ids))
        for i in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                inputs_tensor, hidden, encoder_outputs
            )
            # attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == 2:
                break
        zh_tokens = tokenizer.decode_zh(inputs)
        translated = ' '.join(zh_tokens)
    return translated, en_tokens, inputs

In [44]:
tokenizer.decode_en([36, 4913, 148, 32, 48, 25, 848, 5166, 16262, 16244, 911, 161, 25, 1211, 9, 2516, 25, 690, 141, 443, 9, 1375, 9424, 41, 4986, 781, 225, 401, 1069, 16257, 194, 269, 1282, 3171, 2307, 41, 9424, 16257, 37, 3722, 94, 5325, 1454, 120, 229, 48, 16262, 34, 27, 280, 26, 9, 4206, 120, 35, 3526, 10401, 6121, 232, 504, 9, 3266, 16259])

"I encourage all of you to read Steve's book or to watch the movie to understand not only the beautiful bond that formed between these two men, but how music helped shape that bond, and ultimately was instrumental -- if you'll pardon the pun -- in helping Nathaniel get off the streets."

In [65]:
index = 872
sentence = test_data[index]['translation']['en']
expected_translation = test_data[index]['translation']['zh']

t,en_tokens, zh_tokens = translate_sentence(test_data[index]['translation']['en'],model,device)

print(sentence)
print(expected_translation)
print(t)
print(en_tokens)
print(zh_tokens)

And it did, and we created magic again,  and we had the same result with an audience --  although we did make a little more money on that one.
结果它奏效了。我们再次创造了魔法。 我们再次从观众那里得到了同样的回馈。 不过那部电影的票房要稍微好一点。
的 创 新 ,     ⁇   , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,
[76, 61, 339, 16257, 37, 50, 1539, 2990, 598, 16257, 37, 50, 245, 9, 517, 1387, 108, 98, 2157, 120, 3254, 50, 339, 355, 6, 423, 248, 938, 70, 41, 182, 16259]
[1, 12745, 14773, 14771, 0, 14773, 14773, 14773, 14773, 14773, 1477

In [31]:
len(test_data['translation'])

8549

In [32]:
import pandas as pd
import json

CSV_DIR = f"/"
en = []
zh = []
pred = []

for i in range(0,len(test_data['translation'])):
    en.append(test_data[i]['translation']['en'])
    zh.append(test_data[i]['translation']['zh'])
    t,s,a = translate_sentence(test_data[i]['translation']['en'],model,device)

    pred.append(t)

df = pd.DataFrame()
df['en'] = en
df['zh'] = zh
df['pred'] = pred

df.to_csv(f"{CSV_DIR}230k.csv")


In [35]:
import pandas as pd
import json


csv_name = f"230k.csv"

df = pd.read_csv(CSV_DIR + csv_name)

predicted_list = df['pred'].tolist()

json_data = {
    "predicted": predicted_list
}


json_name = "230k-base-lstm"
with open(f'{CSV_DIR}{json_name}.json', 'w', encoding='utf-8') as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=2)